# credit score

The goal of this project is to support decision making for a banck, by analyzing a csv file that contains personal information of their clients. The banck wants to know what variables they need to consider while deciding if to lend a loan to a client or not. To help the banck, I need to find the correlation between the clients personal data and their ability to pay it back.  

In [1]:
import pandas as pd    #import pandas                                        
import numpy as np
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')

try:
    df = pd.read_csv('credit_scoring_eng.csv') #read and the bank file and save it as df
    
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv') #in case it doesn't work

In [2]:
df.info() # Loking for info on the file measurements. By doing so, I can allredy notice missing values in 2 columns, totla income and days employed.The number of missing values seems symmetrical, so I assume they're Connected to each other. 

df.head(10) #Printed the first 10 rows of the file. First thing I see is that the days_employed clomn has negetive days and floting numbers. ps: I have no idea how this happened. The negative numbers might be a typo?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [3]:
print(df[df['days_employed'].isnull()].count())# I wanted to be sure of my hypothesis, so I used the count with isnull to see if they had the same number of missing values.  
print(df[df['total_income'].isnull()].count()) 

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [4]:
#Now that I see that they have the same number of entries missing, I'll see if it happens simultaneously in a table made of of the missing values of the days_employed column.
(df[df['days_employed'].isnull()].head(10)) # It seems so

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


In [5]:
df[df['days_employed'].isnull()].shape # We can see that the number of rows in the new table is equal to the number of missing values

(2174, 12)

In [6]:
# From this we can conclude that the missing values in both columns are of the same costumers 

In [7]:
# Let's see if the missing data makes up a large percentage of the data. That way we can specify whether we need to fix it or delete it.

num_of_nan = df['total_income'].isnull().sum() #To do so, I sumed up the amount of nun we have in a column.
 
num_of_data = df['purpose'].count() # And the number of rows in the data file. 

precent_of_nan = num_of_nan / num_of_data # then devided one by the over.

print(f"{precent_of_nan:.0%}")

10%


In [ ]:
#We see that the percentage is very significant. It means I have to fill in the blanks.
# Before I did that, I wanted to see if it was a systemic thing.

In [ ]:
#Now I am going category by category, and looking for some kind of relationship between one of the categoris and the missing values.

In [8]:
print(df[df['total_income'].isnull()]['children'].value_counts())
df['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
print(df[df['total_income'].isnull()]['dob_years'].value_counts())
df['dob_years'].value_counts()

34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
39    51
50    51
43    50
45    50
49    50
51    50
29    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
32    37
64    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
71     5
69     5
20     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64


35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [10]:
print(df[df['total_income'].isnull()]['education'].value_counts())
df['education'].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64


secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [11]:
print(df[df['total_income'].isnull()]['family_status'].value_counts())
df['family_status'].value_counts()

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [12]:
print(df[df['total_income'].isnull()]['gender'].value_counts())
df['gender'].value_counts()

F    1484
M     690
Name: gender, dtype: int64


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [13]:
print(df[df['total_income'].isnull()]['income_type'].value_counts())
df['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64


employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [14]:
print(df[df['total_income'].isnull()]['debt'].value_counts())
df['debt'].value_counts()

0    2004
1     170
Name: debt, dtype: int64


0    19784
1     1741
Name: debt, dtype: int64

In [15]:
print(df[df['total_income'].isnull()]['purpose'].value_counts())
df['purpose'].value_counts()

having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
purchase of the house for my family         71
transactions with my real estate            71
transactions with commercial real estate    70
housing renovation                          70
buy commercial real estate                  67
buying property for renting out             65
property                                    62
real estate transactions                    61
buy residential real estate                 61
housing                                     60
building a property                         59
cars                                        57
going to university                         56
to become educated                          55
second-hand car purchase                    54
buying my own

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In [16]:
#If we go through the different categories and parallel their layout to the missing values, you can see that there is no big difference between them.
#From here we can conclude that the missing values have nothing to do with one of the categories specifically
# So I want to see if it's because of duplicates.

In [17]:
df.duplicated().sum()# used duplicated().sum() to see the number of duplicates in the table

54

In [18]:
# The number of duplicates is not large enough to be a factor in our missing values.
# I just need to conclude that maybe ther was a mistake whille uploading the data. 

In [19]:
# Before I can fill in the missing values, I need to corect a number of things in the days_employed column. 
# Most of the days are in negativ values, and some have amounts that are imposible.

In [20]:
def positive_value(row):
    
    days = row['days_employed']
    
    if days < 0:
        return days*(-1)
    else:
        return days

In [21]:
df['days_employed'] = df.apply(positive_value, axis=1)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [ ]:
#To correct the negative values, I created a function that takes values and returns a positive version of the value.
#I aplied it to the "days_employed" column 

In [22]:
def big_num(row):
    
    days = row['days_employed']
    
    if (days / 365) > 46:
        if days/(365*46) < 10:
            return days/10
        return days/100
    return days

In [23]:
df['days_employed'] = df.apply(big_num, axis=1)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [ ]:
# I assumed there were large numbers because of a misspelly, I also assume it could be because they put the decimal point in the wrong place.
#To fix this, I created a function that takes the number of days and divides it by the number of days within a year. 
#If the solution is greater than the maximum time you can be employed (a.k.a. 46 years), I divide the number by 10 or 100 depending on how big it is.
#I took the function and applied it to the "days_employed" column.

In [24]:
df['education'] = df['education'].str.lower()
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [ ]:
# The "education" had some values that were writen in big letters, so I used the ".str.lower()" on the column to turn all the letters to small letters.

In [25]:
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [26]:
def positive_kids(row):
    
    kids = row['children']
    
    if kids < 0:
        return kids*(-1)
    else:
        return kids

In [27]:
df['children'] = df.apply(positive_kids, axis=1)
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [28]:
def real_kids(row):
    
    kids = row['children']
    
    if kids > 18:
        return kids/10
    return kids

In [29]:
df['children'] = df.apply(real_kids, axis=1)
df['children'].value_counts()

0.0    14149
1.0     4865
2.0     2131
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

In [ ]:
#The "kids" column had a problem.
#It had negative values, so I used the "positive_kids" function and threw it on the page.
#They had some who had 20 kids... I decided they meant 2 kids, so I used the "real_kids" function and aplied it to the column.

In [30]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [31]:
df[(df['dob_years'] < 18) | (df['dob_years'] > 120)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0.0,3465.416189,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0.0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3.0,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0.0,3978.565650,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0.0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0.0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0.0,3387.348685,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0.0,3317.412715,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2.0,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


In [ ]:
#In the "dob_years" column, I wanted to see if there were people outside of the understandable age (120-18).
#It looks like this. I found 101 lines out of logical range.

In [32]:
fites_men = df[(df['dob_years'] < 18) | (df['dob_years'] > 120)]['dob_years'].count()
normal_men = df['dob_years'].count()
fites_prcent = fites_men / normal_men
print(f'{fites_prcent:.0%}')

0%


In [ ]:
# I wanted to check the percentage of people out of range. Very close to 0%.

In [33]:
avr_age = df[(df['dob_years'] > 18) & (df['dob_years'] < 120)]['dob_years'].median()
avr_age

43.0

In [34]:
def forgat_your_id(row):

    age = row['dob_years']
    
    if (age < 18) or (age > 120):
        return 43
    return age

In [35]:
df['dob_years'] = df.apply(forgat_your_id, axis=1)
df[(df['dob_years'] < 18) | (df['dob_years'] > 120)]['dob_years'].count()

0

In [ ]:
# To fix the problem I serched for the median age in the between 18-120.
# Created a function named "forgat_your_id" that converts a value out of range to the median.
# Took the function and aplied it to the "dob_years" column.

In [36]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0.0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


In [ ]:
# I went back to see the categories for gender, I wanted to see what XNA was about... I decided it was a trans.

In [37]:
df.duplicated().sum()

71

In [38]:
df = df.drop_duplicates()

In [39]:
df.duplicated().sum()

0

In [40]:
df.shape

(21454, 12)

In [ ]:
# First, I've seen how many duplicates there are after all the actions I've done. The number of duplicates seems to have increased.
# Because of the lack of connection between duplicates and columns, I decided to delete them.

In [41]:
def age_category(row):
    
    age = row['dob_years']
    
    if (18 < age) and (age < 64):
        return 'work force'
    else:
        return 'retired'

In [42]:
df['age_groupe'] = df.apply(age_category, axis=1)
df.head(40)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groupe
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,work force
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,work force
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,work force
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,work force
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,work force
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,work force
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,work force
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,work force
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,work force
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,work force


In [43]:
no_nan_df = df.dropna()

In [44]:
no_nan_df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groupe
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,work force
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,work force
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,work force
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,work force
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,work force
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,work force
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,work force
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,work force
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,work force
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,work force


In [45]:
df['total_income'] = df.groupby(['education','income_type'])['total_income'].transform(lambda grp: grp.fillna(np.mean(grp)))

In [46]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groupe
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102000,purchase of the house,work force
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802000,car purchase,work force
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752000,purchase of the house,work force
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568000,supplementary education,work force
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572000,to have a wedding,work force
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170000,purchase of the house,work force
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156000,housing transactions,work force
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829000,education,work force
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093000,having a wedding,work force
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150000,purchase of the house for my family,work force


In [ ]:
# To deal with NaN values in "total_income" I decided to feel it with the mean of "income_tipe" and "education" in relation to "total_income"
# It seems like the most sensible thing to do, education and work determine people's incomes. 

In [47]:
print(df['total_income'].count())
print(df['family_status'].count())

21454
21454


In [ ]:
# I checked that "total_income" has the same number of values as the other categories 

In [48]:
df['days_employed'] = df.groupby('dob_years')['days_employed'].transform(lambda grp: grp.fillna(np.mean(grp)))

In [ ]:
# For the missing values in "days_employed," I entered the mean of this category relative to "dob_years."
# I did it because it made sense that the older you were, the more opportunity you had to work.

In [49]:
kids_do_debt = df.groupby('children')['debt'].value_counts()
kids_do_debt

children  debt
0.0       0       13028
          1        1063
1.0       0        4410
          1         445
2.0       0        1926
          1         202
3.0       0         303
          1          27
4.0       0          37
          1           4
5.0       0           9
Name: debt, dtype: int64

In [97]:
children_debt_table = pd.pivot_table(df,values = ['debt'],index = ['children'])
children_debt_table.sort_values('debt',ascending = True)

,debt
children,
5.0,0.000000
0.0,0.075438
3.0,0.081818
1.0,0.091658
2.0,0.094925
4.0,0.097561


In [50]:
family_do_debt = df.groupby('family_status')['debt'].value_counts()
family_do_debt

family_status      debt
civil partnership  0        3763
                   1         388
divorced           0        1110
                   1          85
married            0       11408
                   1         931
unmarried          0        2536
                   1         274
widow / widower    0         896
                   1          63
Name: debt, dtype: int64

In [96]:
family_debt_table = pd.pivot_table(df,values = ['debt'],index = ['family_status'])
family_debt_table.sort_values('debt',ascending = True)

,debt
family_status,
widow / widower,0.065693
divorced,0.071130
married,0.075452
civil partnership,0.093471
unmarried,0.097509


In [56]:
education_do_debt = df.groupby('education')['debt'].value_counts()
education_do_debt

education            debt
bachelor's degree    0        4972
                     1         278
graduate degree      0           6
primary education    0         251
                     1          31
secondary education  0       13808
                     1        1364
some college         0         676
                     1          68
Name: debt, dtype: int64

In [98]:
education_debt_table = pd.pivot_table(df,values = ['debt'],index = ['education'])
education_debt_table.sort_values('debt',ascending = True)

,debt
education,
graduate degree,0.000000
bachelor's degree,0.052952
secondary education,0.089902
some college,0.091398
primary education,0.109929


In [ ]:
# Took the Columns 'Education', 'family_status' and 'Children', and collected the percentage of likelihood in each category of being in debt.  

In [51]:
words = ['wedding','house','car','education','estate','property','university']

for word in words:
    print('{} after stemming - {}'.format(word, english_stemmer.stem(word)))

wedding after stemming - wed
house after stemming - hous
car after stemming - car
education after stemming - educ
estate after stemming - estat
property after stemming - properti
university after stemming - univers


In [52]:
def new_purpose(row):
    
    purposes = row['purpose']
    
    for purpose in purposes.split(" "):
        
        stemmed_word = english_stemmer.stem(purpose)
        if ((stemmed_word == 'hous') or (stemmed_word == 'properti')):
            return('housing')
        if (stemmed_word == 'wed'):
            return ('wedding')
        if (stemmed_word == 'estat'):
            return ('real estate')
        if ((stemmed_word == 'educ') or (stemmed_word == 'univers')):
            return ('education')
        if (stemmed_word == 'car'):
            return ('car')

In [53]:
df['purpose'] = df.apply(new_purpose, axis=1)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groupe
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102000,housing,work force
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802000,car,work force
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752000,housing,work force
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568000,education,work force
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572000,wedding,work force
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170000,housing,work force
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156000,housing,work force
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829000,education,work force
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093000,wedding,work force
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150000,housing,work force


In [ ]:
# I also wanted to record the percentage for the Target column. But we have many categories.
# So I decided to frame all the categories in the column and put them in a more general category.
# To do this, I used the stemming of words to create new categories.
# And then I created a function that takes the same words and uses them to catalog all the words in a column.
# Took the function and aplied it on the column "puorpose".

In [63]:
df['purpose'].value_counts()

housing        6347
real estate    4464
car            4306
education      4013
wedding        2324
Name: purpose, dtype: int64

In [99]:
purpose_debt_table = pd.pivot_table(df,values = ['debt'],index = ['purpose'])
purpose_debt_table.sort_values('debt',ascending = True)

,debt
purpose,
housing,0.070269
real estate,0.075269
wedding,0.080034
education,0.092200
car,0.093590


In [54]:
print(df['total_income'].max()) 
df['total_income'].min()

362496.645


3306.762

In [58]:
def income_level(row):
    
    mony = row['total_income']
    
    if (mony > 100000):
        return 'filthy_rich'
    if ((mony < 100000) and (mony > 50000)):
        return 'rich'
    if ((mony < 50000) and (mony > 20000)):
        return 'life is good'
    if ((mony < 20000) and (mony > 10000)):
        return 'financially stable'
    if (mony < 10000):
        return 'do youre best'

In [59]:
df['income_status'] = df.apply(income_level, axis=1)

In [60]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groupe,income_status
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102000,housing,work force,life is good
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802000,car,work force,financially stable
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752000,housing,work force,life is good
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568000,education,work force,life is good
4,0.0,3402.660720,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572000,wedding,work force,life is good
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170000,housing,work force,life is good
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156000,housing,work force,life is good
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829000,education,work force,life is good
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093000,wedding,work force,financially stable
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150000,housing,work force,life is good


In [ ]:
# Wanted to do the same for the "total_income" column. But there were many numbers.
# So I created categories in the function, and each had a variety of revenues that defined it.
# Created a column called "income_statuse" in which I applied the function, using the revenue in "total_income" 

In [62]:
income_do_debt = df.groupby('income_status')['debt'].value_counts()
income_do_debt

income_status       debt
do youre best       0         868
                    1          58
filthy_rich         0          93
                    1           6
financially stable  0        5899
                    1         550
life is good        0       11717
                    1        1041
rich                0        1136
                    1          86
Name: debt, dtype: int64

In [101]:
income_debt_table = pd.pivot_table(df,values = ['debt'],index = ['income_status'])
income_debt_table.sort_values('debt',ascending = True)

,debt
income_status,
filthy_rich,0.060606
do youre best,0.062635
rich,0.070376
life is good,0.081596
financially stable,0.085285


# Summaries:

1.In the "days_employed" column I changed all the negative values to positive. Thats becouse you can't work a negative amount of days. I did that by creating a function that identifise negative values and turns them in to positive ones, and then applied them on the column using the apply method.    

2.I replaced all the values in the "days_employed" column that had values that exceeded the possible amount of years that a client can work. I replaced them by creating a function, that recognizes values above the reasonable range, took the original value and divided it by 10 or 100, depending on how big it was. Then applied the function on the column using the apply method.    

3.In the "education" column, I replaced all the values that had capitall letters with lower case letters, in case of duplications. I did that using the str.lower method.

4.In the "children" column, I got read of all the values that were negative. No one owes the bank a kid.I did that by creating a function that identifise negative values and turns them in to positive ones, and then applied them on the column using the apply method.    

5.There were some values with to many kids in the "children" column. So I created a function that identifis an unreasonable amount of kids and divides that amount by 10. Then applied them on the column using the apply method.    

6.In the "dub_years" column I found that there were values outside of the expected range for a client. To fix that, I found the average age for a client by applying the mean() methode on the reasonble range for age in the column. After that, I created a function that identified all the values outside the range, and replaced them with the mean  value I found.  Then applied them on the column using the apply method.    

7.Because of the lack of connection between duplicates and specific columns, I decided to delet them by using thedrop_duplicates() methode.

8.after seeing no special connection between the missing values and the all the columns, by using the isnull() method with sum() on every column, I decided to fill "days_employed" and "total_income" missing values with the columns mean() in relation with the columns I found had the bigest impact on those columns. To do so, I used the groupby() method with the columns and the transform() methode.

9.I used stemming method in a function to categorise all the difrent values in the "purpose".I did that so it would be easier to evaluet the debt rate in relation to the purpose of the client for asking for a loan. Afterwards I applied the new categoris as the "purpose" column new values, using the apply() method. 

10.I created a function that takes the values of the "total_income" column, and divides it to diferent income categories. I did that, so it would be easier to define the debt rate acording to income. Then applied them on the column using the apply method. 

11.I took all the relevant columns and showed the debt prcentage of the clients depending on their defrent categoris. I did that by creating a "pivot_table" taking the "debt" values and the columns as index. This was performed on the "income_level","children", "purpose" and "education. this enabels me to reach the final conclusions.  



# conclusions

1.There is no direct connection between the number of children and being in debt.
We see this by the fact that debt doesn't really change between the different categories, unless you don't have kids. 
So if the client doesn't have kids, it's optimal. But after the first one, it's not a big change.


2.When it comes to family status, it seems that if the client has not married, his chances of being in debt are highest.
That's probably because these are young people who haven't had time to save enough money yet.
The second highest is the civil partnership. Looks like it's better to get married.
After a wedding, the chances of debt really go down. It should be reasonable that these are people on the older side of the scale, who have had time to save and make a living.

3.In income_level we see that the customer's chances of being in debt do not change according to the level of income.
The percentage of people in debt in the filthy_rich category is the same as the do youre best category.
It probably stems from the fact that the richer you are, the more you like to spend.

4.And when it comes to "purpose," it seems that people who ask for money to buy housing and real estate are the safest.
It could be because people use these things to make more money by renting.
Next is a wedding and after that is a car and education. It's probably caused by young people looking for these things. 